#### Paphos Weather

This page shows various metrics for the weather in Paphos, Cyprus.

Weather data is purchased from Openweather Marketplace [here](https://home.openweathermap.org/marketplace/my_orders)

Download this weather data from here https://drive.google.com/file/d/14GgJu7YfZulZU9_xpS9GVcykdb1nEaUV/view?usp=sharing




In [1]:
def calculate_heat_index(temperature, relative_humidity):
    heat_index = (-8.784695 +
                  1.61139411 * temperature +
                  (2.338549 * relative_humidity) +
                  (-0.14611605 * temperature * relative_humidity) +
                  (-0.01230809 * (temperature ** 2)) +
                  (-0.01642482 * (relative_humidity ** 2)) +
                  (0.00221173 * (temperature ** 2) * relative_humidity) +
                  (0.00072546 * temperature * (relative_humidity ** 2)) +
                  (-0.000003582 * (temperature ** 2) * (relative_humidity ** 2)))
    return heat_index

In [2]:

from datetime import date
from datetime import datetime
import time
import pandas as pd

def toDate(epoch_time):
    return datetime.fromtimestamp(epoch_time)  

In [3]:
import pandas as pd

df=pd.read_json('/Users/walkerrowe/Documents/hypatia/paphosWeather2023.json')

In [4]:
df.columns

Index(['dt', 'dt_iso', 'timezone', 'main', 'clouds', 'weather', 'wind', 'lat',
       'city_name', 'lon', 'rain'],
      dtype='object')

In [5]:
df['dt'].map(toDate)

0       2018-07-01 03:00:00
1       2018-07-01 04:00:00
2       2018-07-01 05:00:00
3       2018-07-01 06:00:00
4       2018-07-01 07:00:00
                ...        
43843   2023-07-01 22:00:00
43844   2023-07-01 23:00:00
43845   2023-07-02 00:00:00
43846   2023-07-02 01:00:00
43847   2023-07-02 02:00:00
Name: dt, Length: 43848, dtype: datetime64[ns]

We have to convert the epoch time to ISO Date, which is something we can read.

The map function runs a function over a series.  A series means a dataframe with one column.  When we write df['new column'] = df['some column'].map(somefunction) it adds the new columns to the dataframe.


In [6]:
df['dateTime']=df['dt'].map(lambda l: toDate(l).strftime("%Y-%m-%d %H:%M:%S"))

In [7]:
df['month']=df['dt'].map(lambda l: toDate(l).strftime("%m"))
df['day']=df['dt'].map(lambda l: toDate(l).strftime("%d"))
df['year']=df['dt'].map(lambda l: toDate(l).strftime("%y"))
df['hour']=df['dt'].map(lambda l: toDate(l).strftime("%H"))
df['hour']=pd.to_numeric(df['hour'])

In [8]:
df['temp']=df["main"].map(lambda x: x["temp"] )
df['temp_min']=df["main"].map(lambda x: x["temp_min"] )
df['temp_max']=df["main"].map(lambda x: x["temp_max"] )
df['humidity']=df["main"].map(lambda x: x["humidity"])

In [9]:
df['heatIndex'] = df.apply(lambda row: calculate_heat_index(row['temp'], row['humidity']), axis=1)


In [10]:

summer=df.loc[df['month'].isin(['06','07','08'])]
afternoon=df.loc[(df['hour'] > 13)]

In [11]:
hot=afternoon[['year','month','day', 'hour' ,'temp','temp_min','temp_max','hour', 'humidity', 'heatIndex']]

We want to caculate the average temperature by date usign the mean() function.  But the problem is that the date is an object inside the **main** object.  See below how to fix that.

In [28]:
hot.sort_values(by='year',ascending=False).head(15)

,year,month,day,hour,temp,temp_min,temp_max,hour,humidity,heatIndex
43844,23,07,01,23,24.68,24.19,25.33,23,83,25.254392
40984,23,03,04,18,15.38,14.68,16.13,18,68,26.596512
40916,23,03,01,22,17.86,14.29,19.11,22,59,25.649337
40917,23,03,01,23,17.59,13.74,18.73,23,59,25.848200
40932,23,03,02,14,19.58,17.72,23.18,14,52,25.257899
40933,23,03,02,15,19.40,17.54,22.49,15,59,24.809754
40934,23,03,02,16,19.17,17.44,22.16,16,58,25.001593
40935,23,03,02,17,17.88,17.22,18.74,17,65,24.812187
40936,23,03,02,18,18.18,17.54,18.76,18,63,24.929373
40937,23,03,02,19,18.54,15.96,20.14,19,53,25.704454


In [21]:
years=hot.groupby('year')

In [26]:
average_colums=years[['temp','temp_min','temp_max','humidity','heatIndex']]

In [29]:
average_colums.mean().sort_values(by='temp',ascending=False)

,temp,temp_min,temp_max,humidity,heatIndex
year,,,,,
18,23.341549,21.734875,24.696402,70.494022,27.287881
21,20.943836,19.458189,22.486562,64.325205,26.884295
20,20.825429,19.718128,21.703730,68.579508,27.368743
19,20.783422,19.463855,21.418340,69.351781,26.848660
22,20.413556,18.831203,22.211145,65.733973,26.645908
23,17.647973,16.080297,19.010621,66.026374,26.033007
